In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, roc_curve, roc_auc_score,log_loss,classification_report
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, LabelEncoder, OneHotEncoder,MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LogisticRegression, LinearRegression
from sklearn.compose import make_column_transformer, make_column_selector
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC



In [2]:
hr = pd.read_csv("HR_comma_sep.csv")

In [3]:
hr.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,Department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.10,0.77,6,247,4,0,1,0,sales,low
3,0.92,0.85,5,259,5,0,1,0,sales,low
4,0.89,1.00,5,224,5,0,1,0,sales,low


In [4]:
y = hr['left']
X = hr.drop('left',axis=1)


In [5]:
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False).set_output(transform = 'pandas')
ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = object) ),
                                  (ohe,make_column_selector(dtype_include = object) ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')
svm = SVC(kernel='linear', probability = True,random_state = 24)
kfold = StratifiedKFold(n_splits = 5,shuffle = True,random_state = 24)
params = {'C':np.linspace(0.001,5,10) }



In [6]:
pipe = Pipeline([('CT',ct),('SVM',svm)])

In [7]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3,stratify=y)

In [8]:
#svm = SVC(kernel='linear', probability = True,random_state = 24)
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.7703934207601689


## with scaling

In [10]:
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False).set_output(transform = 'pandas')
ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = object) ),
                                  (ohe,make_column_selector(dtype_include = object) ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')
scaler_std = StandardScaler()
scaler_mm = MinMaxScaler()
svm = SVC(kernel='linear', probability = True,random_state = 24)
kfold = StratifiedKFold(n_splits = 5,shuffle = True,random_state = 24)
params = {'C':np.linspace(0.001,5,10) }



In [11]:
pipe = Pipeline([('CT',ct),('SCL',scaler_mm),('SVM',svm)])

In [12]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3,stratify=y)

In [13]:
#svm = SVC(kernel='linear', probability = True,random_state = 24)
pipe.fit(X_train,y_train)
y_pred = pipe.predict(X_test)
print(accuracy_score(y_test,y_pred))

0.7741720382307179


In [14]:
y_pred_prob = pipe.predict_proba(X_test)[:,1]
print(roc_auc_score(y_test,y_pred_prob))

0.8002819546310604


## With GridSearchCV

In [16]:
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False).set_output(transform = 'pandas')
ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = object) ),
                                  (ohe,make_column_selector(dtype_include = object) ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')
scaler_std = StandardScaler()
scaler_mm = MinMaxScaler()
svm = SVC(kernel='linear', probability = True,random_state = 24)
kfold = StratifiedKFold(n_splits = 5,shuffle = True,random_state = 24)
params = {'SVM__C':np.linspace(0.001,5,10) }

pipe = Pipeline([('CT',ct),('SCL',scaler_mm),('SVM',svm)])

In [17]:
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=24,test_size=0.3,stratify=y)

In [18]:
gcv = GridSearchCV(pipe,param_grid = params, cv = kfold, scoring = 'roc_auc',verbose = 3)
gcv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END ......................SVM__C=0.001;, score=0.809 total time=  13.2s
[CV 2/5] END ......................SVM__C=0.001;, score=0.808 total time=  12.8s
[CV 3/5] END ......................SVM__C=0.001;, score=0.794 total time=  13.2s
[CV 4/5] END ......................SVM__C=0.001;, score=0.805 total time=  12.6s
[CV 5/5] END ......................SVM__C=0.001;, score=0.792 total time=  13.0s
[CV 1/5] END .........SVM__C=0.5564444444444444;, score=0.814 total time=  12.8s
[CV 2/5] END .........SVM__C=0.5564444444444444;, score=0.815 total time=  12.5s
[CV 3/5] END .........SVM__C=0.5564444444444444;, score=0.799 total time=  12.6s
[CV 4/5] END .........SVM__C=0.5564444444444444;, score=0.810 total time=  12.7s
[CV 5/5] END .........SVM__C=0.5564444444444444;, score=0.804 total time=  12.3s
[CV 1/5] END .........SVM__C=1.1118888888888887;, score=0.815 total time=  13.8s
[CV 2/5] END .........SVM__C=1.1118888888888887;

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000021755C46F60>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._colu...000002174FD84B90>)],
                                                          verbose_feature_names_out=False)),
                                       ('SCL', MinMaxScaler()),
                                       ('SVM',
                                        SVC(kernel='linear', probability=True,
                                            random_state=24))]),
             param_grid={'SVM__C': array([1.00000000e-03, 5.56444444e-01, 1.11188889e+00, 1.66733333e+00,
       2.22277778e+00, 2.77822222e+00, 3.33366667e+00, 3.88911111e+00,
       4.44455556e+00, 5.00000000e+00])},
             scoring='roc_auc', verbose=3)

In [19]:
gcv.best_estimator_

Pipeline(steps=[('CT',
                 ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000021755C76A50>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000021755C779B0>)],
                                   verbose_feature_names_out=False)),
                ('SCL', MinMaxScaler()),
                ('SVM',
                 SVC(C=5.0, kernel='linear', probability=True,
                     random_state=24))])

In [45]:
print(gcv.best_params_)
print(gcv.best_score_)

{'SVM__C': 5.0}
0.8091014936351753


## Polynomial Kernel

In [47]:
ohe = OneHotEncoder(handle_unknown = 'ignore', sparse_output = False).set_output(transform = 'pandas')
ct = make_column_transformer(('passthrough',make_column_selector(dtype_exclude = object) ),
                                  (ohe,make_column_selector(dtype_include = object) ),
                                  verbose_feature_names_out = False).set_output(transform = 'pandas')
scaler_std = StandardScaler()
scaler_mm = MinMaxScaler()
svm = SVC(kernel='poly', probability = True,random_state = 24)
kfold = StratifiedKFold(n_splits = 5,shuffle = True,random_state = 24)
params = {'SVM__C':np.linspace(0.001,5,10),'SVM__degree':[2,3] }

pipe = Pipeline([('CT',ct),('SCL',scaler_mm),('SVM',svm)])

In [49]:
gcv = GridSearchCV(pipe,param_grid = params, cv = kfold, scoring = 'roc_auc',verbose = 3)
gcv.fit(X,y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits
[CV 1/5] END .......SVM__C=0.001, SVM__degree=2;, score=0.849 total time=  28.8s
[CV 2/5] END .......SVM__C=0.001, SVM__degree=2;, score=0.838 total time=  27.4s
[CV 3/5] END .......SVM__C=0.001, SVM__degree=2;, score=0.841 total time=  28.6s
[CV 4/5] END .......SVM__C=0.001, SVM__degree=2;, score=0.827 total time=  28.8s
[CV 5/5] END .......SVM__C=0.001, SVM__degree=2;, score=0.848 total time=  28.7s
[CV 1/5] END .......SVM__C=0.001, SVM__degree=3;, score=0.850 total time=  28.5s
[CV 2/5] END .......SVM__C=0.001, SVM__degree=3;, score=0.841 total time=  28.7s
[CV 3/5] END .......SVM__C=0.001, SVM__degree=3;, score=0.838 total time=  28.9s
[CV 4/5] END .......SVM__C=0.001, SVM__degree=3;, score=0.827 total time=  27.8s
[CV 5/5] END .......SVM__C=0.001, SVM__degree=3;, score=0.844 total time=  29.0s
[CV 1/5] END SVM__C=0.5564444444444444, SVM__degree=2;, score=0.962 total time=  19.5s
[CV 2/5] END SVM__C=0.5564444444444444, S

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000021755C174A0>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._colu...],
                                                          verbose_feature_names_out=False)),
                                       ('SCL', MinMaxScaler()),
                                       ('SVM',
                                        SVC(kernel='poly', probability=True,
                                            random_state=24))]),
             param_grid={'SVM__C': array([1.00000000e-03, 5.56444444e-01, 1.11188889e+00, 1.66733333e+00,
       2.22277778e+00, 2.77822222e+00, 3.33366667e+00, 3.88911111e+00,
       4.44455556e+00, 5.00000000e+00]),
                         'SVM__degree': [2, 3]},
             scoring='roc_auc', verbose=3)

In [51]:
gcv.best_estimator_

Pipeline(steps=[('CT',
                 ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000021755C14E30>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000021755D0BDD0>)],
                                   verbose_feature_names_out=False)),
                ('SCL', MinMaxScaler()),
                ('SVM',
                 SVC(C=5.0, kernel='poly', probability=True, random_state=24))])

In [53]:
print(gcv.best_params_)
print(gcv.best_score_)

{'SVM__C': 5.0, 'SVM__degree': 3}
0.9688806457345971


## Radial 

In [66]:
svm = SVC(kernel='rbf', probability = True,random_state = 24)
kfold = StratifiedKFold(n_splits = 5,shuffle = True,random_state = 24)
params = {'SVM__C':np.linspace(0.001,5,3),'SVM__gamma': np.linspace(0.001,5,3) }

In [68]:
pipe = Pipeline([('CT',ct),('SCL',scaler_mm),('SVM',svm)])

In [70]:
gcv = GridSearchCV(pipe,param_grid = params, cv = kfold, scoring = 'roc_auc',verbose = 3)
gcv.fit(X,y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END ....SVM__C=0.001, SVM__gamma=0.001;, score=0.763 total time=  33.1s
[CV 2/5] END ....SVM__C=0.001, SVM__gamma=0.001;, score=0.774 total time=  35.2s
[CV 3/5] END ....SVM__C=0.001, SVM__gamma=0.001;, score=0.757 total time=  33.1s
[CV 4/5] END ....SVM__C=0.001, SVM__gamma=0.001;, score=0.766 total time=  31.8s
[CV 5/5] END ....SVM__C=0.001, SVM__gamma=0.001;, score=0.793 total time=  32.2s
[CV 1/5] END SVM__C=0.001, SVM__gamma=2.5004999999999997;, score=0.929 total time=  35.6s
[CV 2/5] END SVM__C=0.001, SVM__gamma=2.5004999999999997;, score=0.928 total time=  35.7s
[CV 3/5] END SVM__C=0.001, SVM__gamma=2.5004999999999997;, score=0.939 total time=  35.5s
[CV 4/5] END SVM__C=0.001, SVM__gamma=2.5004999999999997;, score=0.923 total time=  37.1s
[CV 5/5] END SVM__C=0.001, SVM__gamma=2.5004999999999997;, score=0.932 total time=  36.7s
[CV 1/5] END ......SVM__C=0.001, SVM__gamma=5.0;, score=0.936 total time=  38.8s
[CV 

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000021755C174A0>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x0000021755C16D80>)],
                                                          verbose_feature_names_out=False)),
                                       ('SCL', MinMaxScaler()),
                                       ('SVM',
                                        SVC(probability=True,
                                            random_state=24))]),
             param_grid={'SVM__C': array([1.0000e-03, 2.5005e+00, 5.0000e+00]),
                         'SVM__gamma': array([1.0000e-03, 2.5005e+00, 5.0000e+00])},
             scoring='roc_auc', verbose=3)

In [72]:
gcv.best_estimator_

Pipeline(steps=[('CT',
                 ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000021767923A10>),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x0000021767923E60>)],
                                   verbose_feature_names_out=False)),
                ('SCL', MinMaxScaler()),
                ('SVM',
                 SVC(C=5.0, gamma=5.0, probability=True, random_state=24))])

In [74]:
print(gcv.best_params_)
print(gcv.best_score_)

{'SVM__C': 5.0, 'SVM__gamma': 5.0}
0.9812581234821035
